# D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?



#### Download and Clean Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

##### There are 1030 samples to train a model on. Because of the few samples, we have to be careful not to overfit the training data.


#### Let's check the dataset for any missing values.

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

There are no any missing value in our columns.

#### Let's describe our dataset

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
predictors = concrete_data.drop('Strength',axis=1)
target = concrete_data['Strength'] # Strength column

#### Check the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Normalize the data by substracting the mean and dividing by the standard deviation.

##### Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


##### Save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors 1 dimension array

In [11]:
n_cols

8

#### Let's import the Keras library

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

#### Build a Neural Network

###### Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [17]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
   
    return model

###### The above function create a model that has One hidden layer of 10 nodes, and a ReLU activation function.Use the adam optimizer and the mean squared error  as the loss function.

##### Randomly split the data into a training and test sets by holding 30% of the data for testing.


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state= 42)

In [19]:
print('Train set:',X_train.shape,y_train.shape)
print('Test set:',X_test.shape,y_test.shape)

Train set: (721, 8) (721,)
Test set: (309, 8) (309,)



#### Train and Test the Network

##### Call the function now to create our model.

##### Train the model on the training data using 50 epochs.

In [20]:
# build the model
model = regression_model()

In [21]:
# fit the model
model.fit(X_train, y_train, epochs= 50 , verbose = 0)

2022-07-24 11:34:10.584778: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-07-24 11:34:10.595744: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593900000 Hz
2022-07-24 11:34:10.596486: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5626a4a490c0 executing computations on platform Host. Devices:
2022-07-24 11:34:10.596534: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-07-24 11:34:10.720371: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

##### Evaluate the model on the test data 


In [22]:
train_mse = model.evaluate(X_train, y_train,verbose=0)
test_mse= model.evaluate(X_test, y_test,verbose=0)
print('Train: %.2f, Test:%.2f' % (train_mse,test_mse))

Train: 79.81, Test:89.09


In [23]:
y_pred = model.predict(X_test)

##### Compute the mean squared error between the predicted concrete strength and the actual concrete strength. 

##### Use the mean_squared_error function from Scikit-learn.

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
MSE = mean_squared_error(y_test, y_pred)
MSE

89.0920196242122

##### Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.



In [26]:
list_mse = []
for i in range(0,50):
    # Randomly split the data 
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    #Train Model, Evaluation and Predict  
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    print('MSE' +str(i+1) +':',(MSE))
    # creat list of mean squared error
    mean_square_error = mean_squared_error(y_test, y_pred)
    list_mse.append(mean_square_error)


MSE1: 65.77008854146914
MSE2: 50.65997432967991
MSE3: 43.950888180038305
MSE4: 43.547991014992924
MSE5: 40.17296320103519
MSE6: 55.70568032866543
MSE7: 49.4180056500975
MSE8: 38.92046417927665
MSE9: 42.26801346343698
MSE10: 38.91240262522281
MSE11: 38.62567436193571
MSE12: 35.18137971945951
MSE13: 36.877723514840824
MSE14: 33.819495111607424
MSE15: 31.078199935962466
MSE16: 36.41120008203204
MSE17: 40.77334431768621
MSE18: 29.363152260147636
MSE19: 26.981974567796033
MSE20: 31.620477904779626
MSE21: 28.66778502726632
MSE22: 32.41346145679264
MSE23: 32.54909173416088
MSE24: 39.00598721056694
MSE25: 36.07974013232876
MSE26: 32.08035362268343
MSE27: 25.070431175355388
MSE28: 32.98959807361985
MSE29: 30.7826265810377
MSE30: 33.78560387818173
MSE31: 34.17646085868761
MSE32: 27.36412107597277
MSE33: 23.52415615686707
MSE34: 29.218204214349146
MSE35: 32.03120318354141
MSE36: 31.095882607123613
MSE37: 33.319551745664725
MSE38: 39.97749137261153
MSE39: 30.896772106874337
MSE40: 26.1493711502420

##### Report the mean and the standard deviation of the mean squared errors.

In [27]:
list_mse = np.array(list_mse)

In [28]:
print('The mean of the mean squared errors:',np.mean(list_mse))
print('The standard deviation of the mean squared errors:',np.std(list_mse))

The mean of the mean squared errors: 34.91237368692156
The standard deviation of the mean squared errors: 7.879133170092283


# How does the mean of the mean squared errors compare to that from Step B?

In [29]:
report = [['B','52.22','14.87'],\
          ['D',"%.2f"%np.mean(list_mse),"%.2f"%np.std(list_mse)]]
df_r=pd.DataFrame(report, columns=['PART','Mean of MSE','SD of MSE'])
df_r

,PART,Mean of MSE,SD of MSE
0,B,52.22,14.87
1,D,34.91,7.88


The mean of the mean squared errors and The standard deviation of the mean squared errors of 
#### D is less than B #### 
when we have three hidden layers .